In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/wikipedia-sentences/wikisent2.txt


In [2]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


/kaggle/input/wikipedia-sentences/wikisent2.txt


In [3]:
import random

# Function to read a text file and return its content
def read_file(file_path):
    with open(file_path, 'r') as file:
        content = file.readlines()
    return content

# Function to shuffle the lines in a file and save to a new file
def shuffle_and_sample(file_path, sample_size=1000, output_file='shuffled_sample.txt'):
    # Read the original file
    lines = read_file(file_path)
    
    # Shuffle the lines
    random.shuffle(lines)
    
    # Take a sample of the shuffled lines
    sample_lines = lines[:sample_size]
    
    # Save the sample to a new file
    with open(output_file, 'w') as file:
        file.writelines(sample_lines)

# File path of the original text file
input_file_path = '/kaggle/input/wikipedia-sentences/wikisent2.txt'

# Call the function to shuffle and sample
shuffle_and_sample(input_file_path, sample_size=1000, output_file='shuffled_sample.txt')


In [4]:
!ls


shuffled_sample.txt


In [5]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer, TextDataset, DataCollatorForLanguageModeling, Trainer, TrainingArguments

# Load the GPT-2 tokenizer and model
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2')


2024-08-08 13:47:26.024330: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-08 13:47:26.024458: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-08 13:47:26.176865: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [6]:
def load_dataset(file_path, tokenizer):
    return TextDataset(
        tokenizer=tokenizer,
        file_path=file_path,
        block_size=128
    )

train_dataset = load_dataset('shuffled_sample.txt', tokenizer)


/opt/conda/lib/python3.10/site-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


In [7]:
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False
)


In [8]:
training_args = TrainingArguments(
    output_dir='./results',
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=4,
    save_steps=10_000,
    save_total_limit=2,
    report_to="none"  # Add this line to disable W&B
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
)


In [9]:
trainer.train()


Step,Training Loss


TrainOutput(global_step=159, training_loss=3.6479979701012186, metrics={'train_runtime': 673.5619, 'train_samples_per_second': 0.944, 'train_steps_per_second': 0.236, 'total_flos': 41545433088000.0, 'train_loss': 3.6479979701012186, 'epoch': 3.0})

In [10]:
trainer.save_model('./gpt2-autocomplete')
tokenizer.save_pretrained('./gpt2-autocomplete')

('./gpt2-autocomplete/tokenizer_config.json',
 './gpt2-autocomplete/special_tokens_map.json',
 './gpt2-autocomplete/vocab.json',
 './gpt2-autocomplete/merges.txt',
 './gpt2-autocomplete/added_tokens.json')

In [11]:
!ls results
!ls results/runs

checkpoint-159
ls: cannot access 'results/runs': No such file or directory


In [12]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer, pipeline

# Define the path to your fine-tuned model and tokenizer
model_dir = "./gpt2-autocomplete"

# Load the fine-tuned model and tokenizer
model = GPT2LMHeadModel.from_pretrained(model_dir)
tokenizer = GPT2Tokenizer.from_pretrained(model_dir)

# Create a text generation pipeline
text_generator = pipeline('text-generation', model=model, tokenizer=tokenizer)

# Generate text
prompt = "Your input text here"
generated_text = text_generator(prompt, max_length=50, truncation=True)
print(generated_text[0]['generated_text'])


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Your input text here is for reference only.
The list could be expanded or expanded by changing the name of the list editor to the editor or using JavaScript.
In November 1969, the International Institute for Strategic Studies (IBS) was founded to


In [13]:
prompts = [
    "The future of artificial intelligence is",
    "In the year 2100, humanity will",
    "The key to solving climate change lies in"
]

for prompt in prompts:
    generated_text = text_generator(prompt, max_length=50, truncation=True)
    print(f"Prompt: {prompt}")
    print(f"Generated Text: {generated_text[0]['generated_text']}\n")


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Prompt: The future of artificial intelligence is
Generated Text: The future of artificial intelligence is at a crossroads when it comes to predicting future behavior that will have serious impacts on the health and safety of humans.
The study presents five major features in the theory of how new devices of the future use human-



Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Prompt: In the year 2100, humanity will
Generated Text: In the year 2100, humanity will have established an empire.
In 2015, a total of 7 billion people in the world were living on 3.5 billion acres of land.
The novel features characters from across both genders with the characters' journeys

Prompt: The key to solving climate change lies in
Generated Text: The key to solving climate change lies in a transition to low-carbon energy sources, such as solar and wind power.
The team used a special type of carbon nanotubes developed at the University of Alabama to create a metal-reinforced



In [14]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# Define the path to your fine-tuned model and tokenizer
model_dir = "./gpt2-autocomplete"

# Load the fine-tuned model and tokenizer
model = GPT2LMHeadModel.from_pretrained(model_dir)
tokenizer = GPT2Tokenizer.from_pretrained(model_dir)

# Create a text generation pipeline
text_generator = pipeline('text-generation', model=model, tokenizer=tokenizer)

# Function to generate next word predictions
def predict_next_word(text, top_k=5):
    inputs = tokenizer(text, return_tensors="pt")
    outputs = model(**inputs)
    predictions = outputs.logits[:, -1, :]
    top_k_predictions = predictions.topk(top_k).indices[0].tolist()
    
    suggested_words = [tokenizer.decode([pred]) for pred in top_k_predictions]
    return suggested_words

# Example usage
text = "The future of artificial intelligence is"
suggested_words = predict_next_word(text)
print("Next word suggestions:", suggested_words)


Next word suggestions: [' in', ' a', ' at', ' uncertain', ' still']


In [15]:
from transformers import pipeline

#Create a text generation pipline
test_generator = pipeline("text-generation", model=model,tokenizer=tokenizer)

# Function to generate next sentence
def predict_next_sentence(text, max_length=50):
    generated_text = text_generator(text, max_length=max_length, num_return_sequences=1)
    full_text = generated_text[0]['generated_text']
    
    # Split the generated text into sentences and return the first complete sentence
    sentences = full_text.split('.')
    if len(sentences) > 1:
        return sentences[0] + '.'
    else:
        return full_text
    
#Ecample usage
text = 'In the year 2100, humanity will'
next_sentence = predict_next_sentence(text)
print("Next sentence suggestion:", next_sentence)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Next sentence suggestion: In the year 2100, humanity will go to a peak of 300 million live births, and 10 million people will live on Earth, with 8.


In [16]:
def writing_assistant():
    print('Welcome to the writing Assisitant')
    print("Start typing your text. type 'exit' to quit.")
    
    context = ""
    while True:
        user_input = input("You: ")
        if user_input.lower() == "exit":
            break
            
        context += " " + user_input
        word_suggestions = predict_next_word(context)
        sentence_suggestion = predict_next_sentence(context)
        
        print("Word suggestion:", word_suggestions)
        print("Sentences suggestion:", sentence_suggestion)
        
writing_assistant()

Welcome to the writing Assisitant
Start typing your text. type 'exit' to quit.


You:  people like to say hi,


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Word suggestion: [' but', ' and', ' or', ' so', ' they']
Sentences suggestion:  people like to say hi, and I had no business in Canada after the 2008 financial crisis, so I played around with that a lot before taking a job.


You:  but i really don't like that


...

Word suggestion: ['ive', '\xa0', 'ike', 'iced', 'lly']
Sentences suggestion:  people like to say hi, but i really don't like that  they like to say hi.


You:  so what do people like to say


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Word suggestion: [' hi', '?', ' to', ' when', ',']
Sentences suggestion:  people like to say hi, but i really don't like that  they like to say hi.


You:  exit
